<a href="https://colab.research.google.com/github/beekal/ChatServerUsingSocket/blob/master/101.%20Basics%20-%20TF/3_TF_Simple_Linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## 1.0 Tensor Flow Basics 
 

In [0]:
###############
#1. Import libraries
###########
import tensorflow as tf

################
# Define Program
##############
# Define X, y
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# define variables
bias = tf.Variable(0.0)

# define prediction function n loss function
y_pred = tf.pow(x, 2) + bias
loss =  tf.abs(y_pred - y )


#############
# Session initialisation
################
sess =  tf.Session()
#run session by initialising all variables
sess.run( tf.initialize_all_variables() )


##############
# Session run
#############
#try:
print(sess.run(loss, {x:  3.0, y :  9.0}))
#except:
#  print('Error in the session code.')

0.0


**2.0  Handling Categorical variables**

In [0]:
import tensorflow as tf

In [0]:
########
# Data
##########
data = {
        'sales': [5,10,8,9],
        'dept' : ['sports', 'sports','gardening','gardening' ]
      }
########
# Tensorflow Feature encoding
###############
dept_col = tf.feature_column.categorical_column_with_vocabulary_list(
          'dept',['sports','gardening'] )
#dept_col = tf.feature_column.indicator_column(dept_col)
col_defn = [ tf.feature_column.numeric_column('sales'),
             tf.feature_column.indicator_column(dept_col) ]

X = tf.feature_column.input_layer( data, col_defn )

##########
# Tensorflow variable intialisation
############
var_init = tf.global_variables_initializer()
tbl_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, tbl_init))

#######
# Run session (Tensorflow)
#######
print(sess.run(X))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


**# 2.1 A simple linear model**

1. No Optimisation

In [0]:
#X = tf.feature_column.input_layer( data, col_defn )
y = tf.constant([ [0],[-1],[-2],[-3] ], dtype=tf.float32)

##Model
linear_model = tf.layers.Dense(units = 1)
y_pred = linear_model(X)


##########
# Tensorflow variable intialisation
############
var_init = tf.global_variables_initializer()
tbl_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, tbl_init))

print(sess.run(y_pred))

[[2.0081606]
 [4.2850294]
 [3.8635886]
 [4.318962 ]]


**# 2.2  Loss Defn**

In [0]:
loss = tf.losses.mean_squared_error(labels= y, predictions= y_pred)

print(sess.run(loss))

29.978281


**# 2.3 Optimisers**

The optmisers act on each variable incrementally changing them to minimise the loss function.

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train = optimizer.minimize(loss)

for i in range(10):
  _,loss_val = sess.run((train, loss))
  print("Iter ",i, '. Loss', loss_val )

Iter  0 . Loss 0.25199348
Iter  1 . Loss 0.218833
Iter  2 . Loss 0.19201347
Iter  3 . Loss 0.17032051
Iter  4 . Loss 0.15277499
Iter  5 . Loss 0.13894626
Iter  6 . Loss 0.18755563
Iter  7 . Loss 10.001505
Iter  8 . Loss 1615.6956
Iter  9 . Loss 264082.7


In [0]:
######
# Scoping variables for low memory footprint
######
sess = tf.InteractiveSession()

def func(w):
  #x = tf.get_variable('x', initializer= tf.constant(5.0, \
  #                               shape = None, dtype=tf.float32 ))
  x = tf.Variable(tf.constant(5.0,shape = None, dtype=tf.float32 ) )
  z = x*w
  return z

#with tf.variable_scope('A', reuse=True):
#  z1 = func(tf.constant(1.0,dtype=tf.float32))


sess = tf.global_variables_initializer()#.run()
sess.run(func(2))
#sess.run([z1])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TypeError: ignored

In [0]:
import tensorflow as tf

# 3.0 Importing CSV Data

tf.placeholders are great for small data and for simple programs but are not suited for large files or production system.

### tf.data.Dataset : For large files

In [0]:
import pandas as pd

house_dt = pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv", sep=",")
house_dt.sample(2)

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
16256    -122.49     37.98                34.0       1256.0           178.0   
16616    -122.71     38.33                13.0       4011.0           936.0   

       population  households  median_income  median_house_value  
16256       460.0       174.0         6.4271            451700.0  
16616      2064.0       914.0         3.6953            157600.0

In [0]:
house_small = house_dt[0:5][['total_rooms','population']] #[0:5]#.sample(5)
house_small

total_rooms  population
0       5612.0      1015.0
1       7650.0      1129.0
2        720.0       333.0
3       1501.0       515.0
4       1454.0       624.0

In [0]:
house_small.reindex([2,3,4,0,1])
#city[['longitude', 'latitude'] ][0:2]

total_rooms  population
2        720.0       333.0
3       1501.0       515.0
4       1454.0       624.0
0       5612.0      1015.0
1       7650.0      1129.0

In [0]:
Epoch = 0 #@param {type:"integer"}
# Reindexing is a excellent way to make sure that no biad feeds into the data
